In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt 
import pandas as pd 
import folium
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import requests
import json
import geocoder
import datetime
from datetime import timedelta
import sys; sys.path.append("../Notebook/")

%time
%matplotlib inline

Wall time: 0 ns


# Dataframe from Velocity

In [2]:
array_files_acidentes_vitimas = glob.glob("../Velocidades Vias 15 min/*.csv")

In [3]:
def concatenateDatasets(array_to_concatenate):
    dataset = 0
    array = []
    for velocidadeVia in array_to_concatenate:
        dataset = pd.read_csv(velocidadeVia,sep=';')
        array.append(dataset)
    return pd.concat(array)

In [4]:
dataFrame = concatenateDatasets(array_files_acidentes_vitimas)

# Dataframe from Red Light Camera

In [5]:
array_files_acidentes_vitimas_lombadas = glob.glob("../Velocidades Vias 15 min/Lombadas/*.csv")

In [6]:
dataFrameLombadas = concatenateDatasets(array_files_acidentes_vitimas_lombadas)

## Cleanning DataFrame ( Velocity and Red Light Camera)

In [7]:
dataFrameTrainVelocity = pd.DataFrame.copy(dataFrame)

In [46]:
dataFrameTrainRedLightCamera = pd.DataFrame.copy(dataFrameLombadas)

In [47]:
dataFrameTrainRedLightCamera['bairro'] = ''

In [48]:
dataFrameTrainRedLightCamera.sort_values("equipamento",inplace=True)

In [49]:
#Removing Duplicates
dataFrameTrainRedLightCamera = dataFrameTrainRedLightCamera.drop_duplicates(subset="equipamento")

In [50]:
dataFrameTrainRedLightCamera = dataFrameTrainRedLightCamera.drop(dataFrameTrainRedLightCamera.index[0:32])

### Removing a sufix from a Equipament column from velocity dataset

In [51]:
def remove_suffix(text, suffix):
    return text[:-len(suffix)] if text.endswith(suffix) and len(suffix) != 0 else text

In [52]:
def remove_rec_string(array_remove):
    array_values = []
    for value in array_remove:
        value = remove_suffix(value,'REC')
        array_values.append(value) 
    return array_values 

In [53]:
dataFrameTrainVelocity['equipamento'] = remove_rec_string(dataFrame['equipamento'])

In [54]:
address='Recife, PE'
geolocater=Nominatim(user_agent="recife_explorer")
location = geolocater.geocode(address)
try:
    latitude = location.latitude
    longitude = location.longitude
except GeocoderTimedOut:
    print("Geocoder Timeout")

### Insert a Red Lights Camera on the Map

In [55]:
districtArray = dataFrameTrainRedLightCamera['bairro']

In [56]:
recife_map=folium.Map(location=[latitude,longitude],zoom_start=10)
i = 0
array = []
for lat,long in zip(dataFrameTrainRedLightCamera['latitude'],dataFrameTrainRedLightCamera['longitude']):
    locationReverse = geolocater.reverse(str(lat)+","+str(long))
    locationDict = locationReverse.raw
    district = locationDict['address']['suburb']
    array.append(district)
    folium.CircleMarker(
                    [lat,long],
                    raduis=0.5,
                    color='',
                    #popup=distric,
                    fill=True,
                    fill_color='red',
                    fill_opacity=0.6,
                    parse_html=True
                    ).add_to(recife_map)
display(recife_map)
dataFrameTrainRedLightCamera['bairro'] = array

In [36]:
districtArray = list(dict.fromkeys(districtArray))

In [37]:
districtArray = sorted(districtArray)

In [58]:
districArray = np.asarray(dataFrameTrainRedLightCamera['bairro'])

### Insert a District on the Map

In [28]:
for value in districtArray:
    location = geolocater.geocode(value+','+'Recife')
    latitude = location.latitude
    longitude = location.longitude
    folium.Circle(
                location = [latitude,longitude],
                radius=800,
                popup=value,
                color='#3186cc',
                fill=True,
                fill_color="#3186cc",
                ).add_to(recife_map)
display(recife_map)

### Process Data

In [29]:
dataframeProcess = pd.DataFrame(columns=['hora','minuto',"dia",'mes','ano','diaDaSemana','rua','bairro','longitude','latitude','velocidade_via'])

In [30]:
def divideInDateDayYear():
    week_days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    i = 0
    arrayDia = []
    arrayMes = []
    arrayAno = []
    arrayDiaDaSemana = []
    for value in dataFrameTrainVelocity['data']:
        splitValue = value.split('-')
        arrayDia.append(splitValue[2])
        arrayMes.append(splitValue[1])
        arrayAno.append(splitValue [0]) 
        week_num = datetime.date(int(splitValue[0]),int(splitValue[1]),int(splitValue[2])).weekday()
        arrayDiaDaSemana.append(week_days[week_num])
    dataframeProcess['dia'] = arrayDia
    dataframeProcess['mes'] = arrayMes
    dataframeProcess['ano'] = arrayAno
    dataframeProcess['diaDaSemana'] = arrayDiaDaSemana

In [31]:
divideInDateDayYear()

In [32]:
dataframeProcess['velocidadeMedia'] = ''
dataframeProcess['numeroVeiculos'] = ''

In [33]:
columnsDataset = dataFrameTrainVelocity.columns

In [34]:
columnsDataset = list(dict.fromkeys(columnsDataset))

In [35]:
del columnsDataset[0:6]

In [197]:
def getIntervalo(name):
    if(name == 'qtd_0a10km'):
        return 5
    if(name == 'qtd_11a20km'):
        return 15.5
    if(name == 'qtd_21a30km'):
        return 25.5
    if(name == 'qtd_31a40km'):
        return 35.5
    if(name == 'qtd_41a50km'):
        return 45.5
    if(name == 'qtd_51a60km'):
        return 55.5
    if(name == 'qtd_61a70km'):
        return 65.5
    if(name == 'qtd_71a80km'):
        return 75.5
    if(name == 'qtd_81a90km'):
        return 80.5
    if(name == 'qtd_91a100km'):
        return 95.5
    if(name == 'qtd_acimade100km'):
        return 110

In [198]:
def setMeanVelocity():
    arrayVelocidadeMedia = []
    arrayQtdCarros = []
    quantidadeTotalCarros = 0
    ponderatedMean = 0
    for index, row in dataFrameTrainVelocity.iterrows():
        for columns in columnsDataset: 
            ponderatedMean =(row[columns] * getIntervalo(columns)) + ponderatedMean
            quantidadeTotalCarros = row[columns] + quantidadeTotalCarros
        veloMedia = ponderatedMean / quantidadeTotalCarros
        arrayVelocidadeMedia.append(veloMedia)
        arrayQtdCarros.append(quantidadeTotalCarros)
        ponderatedMean = 0  
        quantidadeTotalCarros = 0
    dataframeProcess['velocidadeMedia'] = arrayVelocidadeMedia
    dataframeProcess['numeroVeiculos'] = arrayQtdCarros
    return dataframeProcess

In [199]:
setMeanVelocity()

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos,equipamento
0,0,15,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,2,FS002
1,0,30,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,1,FS002
2,0,45,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.400000,5,FS002
3,0,59,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,10.250000,2,FS002
4,1,75,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,4,FS002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,1379,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,22.243243,37,FS037
9561062,23,1395,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,22.057143,35,FS037
9561063,23,1410,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,19.771429,35,FS037
9561064,23,1425,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,19.388889,18,FS037


In [38]:
def setData():
    arrayData = []
    for value in dataFrameTrainVelocity['hora']:
        arrayData.append(str(value))
    dataframeProcess['hora'] = arrayData
    return dataframeProcess

In [39]:
setData()

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos
0,0,NaN,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
1,0,NaN,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
2,0,NaN,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
3,0,NaN,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
4,1,NaN,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,NaN,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561062,23,NaN,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561063,23,NaN,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561064,23,NaN,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,


In [40]:
def setMinutes():
    arrayMinuto = []
    for hora,minuto_intervalo in zip (dataframeProcess['hora'],dataFrameTrainVelocity['minutos_intervalo']):
        splitMinutosIntervalo = minuto_intervalo.split('-')
        delta = timedelta(hours=int(hora), minutes=int(splitMinutosIntervalo[1]))
        total_seconds = delta.total_seconds()
        minutes = int(total_seconds // 60)
        arrayMinuto.append(minutes)
    dataframeProcess['minuto'] = arrayMinuto
    return dataframeProcess

In [41]:
setMinutes()

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos
0,0,15,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
1,0,30,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
2,0,45,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
3,0,59,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
4,1,75,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,1379,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561062,23,1395,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561063,23,1410,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561064,23,1425,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,


In [43]:
dataframeProcess

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos
0,0,15,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
1,0,30,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
2,0,45,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
3,0,59,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
4,1,75,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,1379,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561062,23,1395,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561063,23,1410,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,
9561064,23,1425,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,


In [44]:
def setEquipamento():
    arrayData = []
    for value in dataFrameTrainVelocity['equipamento']:
        arrayData.append(value)
    dataframeProcess['equipamento'] = arrayData
    return dataframeProcess

In [45]:
setEquipamento()

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos,equipamento
0,0,15,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,,FS002
1,0,30,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,,FS002
2,0,45,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,,FS002
3,0,59,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,,FS002
4,1,75,01,01,2016,Friday,NaN,NaN,NaN,NaN,NaN,,,FS002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,1379,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,,FS037
9561062,23,1395,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,,FS037
9561063,23,1410,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,,FS037
9561064,23,1425,31,01,2020,Friday,NaN,NaN,NaN,NaN,NaN,,,FS037


In [46]:
def findValueDataFrame(dataframe,column,valueToFind,columnObject):
    value = dataframe.loc[dataframe[column] == valueToFind]
    valueToFind = value[columnObject].values
    
    if(valueToFind != None):
        return valueToFind[0]
    else:
        return ''

In [185]:
dataframeProcess['latitude'] = ''

In [188]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [193]:
dataframeProcess['latitude'] = dataframeProcess.apply(lambda row: findValueDataFrame(dataFrameTrainRedLightCamera,'equipamento',row['equipamento'],'latitude'),axis=1)

In [196]:
dataframeProcess['longitude'] = dataframeProcess.apply(lambda row: findValueDataFrame(dataFrameTrainRedLightCamera,'equipamento',row['equipamento'],'longitude'),axis=1)
dataframeProcess['velocidade_via'] = dataframeProcess.apply(lambda row: findValueDataFrame(dataFrameTrainRedLightCamera,'equipamento',row['equipamento'],'velocidade_via'),axis=1)
dataframeProcess['bairro'] = dataframeProcess.apply(lambda row: findValueDataFrame(dataFrameTrainRedLightCamera,'equipamento',row['equipamento'],'bairro'),axis=1)
dataframeProcess['rua'] = dataframeProcess.apply(lambda row: findValueDataFrame(dataFrameTrainRedLightCamera,'equipamento',row['equipamento'],'logradouro'),axis=1)

In [200]:
dataframeProcess.to_csv('dfProcess.csv',encoding='utf-8')

In [208]:
dataframeProcess.to_pickle('dfProcess.pickle')

### Train Dataset with MLP Backpropagation

In [12]:
import bz2
import pickle
import _pickle as cPickle

In [14]:
def loosen(file):
 pikd = open(file, 'rb')
 data = pickle.load(pikd)
 pikd.close()
 return data

In [15]:
dfTrain = loosen('dfProcess.pickle')

In [19]:
dfTrain

,hora,minuto,dia,mes,ano,diaDaSemana,rua,bairro,longitude,latitude,velocidade_via,velocidadeMedia,numeroVeiculos,equipamento
0,0,15,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,2,FS002
1,0,30,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,1,FS002
2,0,45,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.400000,5,FS002
3,0,59,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,10.250000,2,FS002
4,1,75,01,01,2016,Friday,"RUA MADRE DE DEUS, SEMAFORO 020.",Bairro do Recife,-34.873669,-8.0634,30 Km/h,15.500000,4,FS002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9561061,22,1379,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,22.243243,37,FS037
9561062,23,1395,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,22.057143,35,FS037
9561063,23,1410,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,19.771429,35,FS037
9561064,23,1425,31,01,2020,Friday,"AV. GOV. AGAMENON MAGALHAES, SEMAFORO 069",Paissandu,-34.89733,-8.06268,60 Km/h,19.388889,18,FS037


In [33]:
import os

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,f1_score,precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from tensorflow.keras.models import Sequential # Foward propagation
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers #Back Propagation

import matplotlib.pyplot as plt

os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz/bin/'

In [5]:
def setDiaDaSemanaInt(diaDaSemana):
    if diaDaSemana == 'Sunday':
        return 1
    if diaDaSemana == 'Monday':
        return 2
    if diaDaSemana == 'Tuesday':
        return 3
    if diaDaSemana == 'Wednesday':
        return 4
    if diaDaSemana == 'Thursday':
        return 5
    if diaDaSemana == 'Friday':
        return 6
    if diaDaSemana == 'Saturday':
        return 7

In [6]:
dfTrain.diaDaSemana = dfTrain.apply(lambda row: setDiaDaSemanaInt(row.diaDaSemana),axis=1)

In [7]:
dfTrain.hora = dfTrain.apply(lambda row: int(row.hora),axis=1)

### Create a dataset by dfTrain to train with NN

In [8]:
dataFrameNN = pd.DataFrame(columns=['seg','ter','quar','qui','sex','sab','dom','velocidadeMedia','hora'])

In [9]:
dataFrameNN.velocidadeMedia = dfTrain.velocidadeMedia

In [10]:
dataFrameNN.hora = dfTrain.hora

In [11]:
dataFrameNN['diaDaSemana'] = dfTrain.diaDaSemana

In [12]:
dataFrameNN.seg = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 2 else 0 ,axis=1)

In [13]:
dataFrameNN.ter = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 3 else 0 ,axis=1)

In [14]:
dataFrameNN.quar = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 4 else 0 ,axis=1)

In [15]:
dataFrameNN.qui = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 5 else 0 ,axis=1)

In [16]:
dataFrameNN.sex = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 6 else 0 ,axis=1)

In [17]:
dataFrameNN.sab = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 7 else 0 ,axis=1)

In [18]:
dataFrameNN.dom = dfTrain.apply(lambda row: 1 if row['diaDaSemana'] == 1 else 0 ,axis=1)

In [19]:
dataFrameNN['isJan'] = ''
dataFrameNN['isJuly'] = ''

In [20]:
dataFrameNN['mes'] = dfTrain.mes

In [20]:
dataFrameNN = pd.read_csv('dataFrameNN.csv')

In [5]:
dataFrameNN = dataFrameNN.drop(['Unnamed: 0'],axis = 'columns')

In [22]:
def isJuly_isJan(mes):
    if(int(mes) == 7 or int(mes) == 1):
        return 1
    else:
        return 0

In [23]:
dataFrameNN['isJan_isJuly'] = dataFrameNN.apply(lambda row: isJuly_isJan(row['mes']),axis=1)

#If is 1 is true, if is 0 is not july or january

In [24]:
dataFrameNN.to_csv('dataFrameNN.csv',encoding='utf-8')

### Train Red light Camera

In [76]:
import numpy as np
import glob
import matplotlib.pyplot as plt 
import pandas as pd 
#import folium
import requests
import json

import datetime
from datetime import timedelta

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,f1_score,precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
%time
%matplotlib inline

Wall time: 0 ns


In [77]:
dfProcess= pd.read_csv('dfProcess.csv')

In [78]:
dfProcess=dfProcess.drop(['Unnamed: 0'],axis='columns')

In [79]:
districArray = removeDuplicated(districArray)

In [80]:
len(districArray)

20

In [81]:
arrayEquipamento =dfProcess['equipamento'].to_numpy()

In [82]:
arrayEquipamento = removeDuplicated(arrayEquipamento)

In [83]:
arrayEquipamento = np.asarray(arrayEquipamento)

In [84]:
arrayEquipamento.sort()

In [86]:
def valueToZeroToOne_velocidadeMedia(value):
    return ((value - min_velocidadeMedia)/(max_velocidadeMedia - min_velocidadeMedia))

In [87]:
def valueToZeroToOne_hora(value):
    return ((value - min_hora)/(max_hora - min_hora))

In [88]:
def setDiaDaSemanaInt(diaDaSemana):
    if diaDaSemana == 'Sunday':
        return 1
    if diaDaSemana == 'Monday':
        return 2
    if diaDaSemana == 'Tuesday':
        return 3
    if diaDaSemana == 'Wednesday':
        return 4
    if diaDaSemana == 'Thursday':
        return 5
    if diaDaSemana == 'Friday':
        return 6
    if diaDaSemana == 'Saturday':
        return 7

In [89]:
dfProcess['diaDaSemana'] = dfProcess.apply(lambda row: setDiaDaSemanaInt(row['diaDaSemana']) ,axis=1)

In [90]:
def isJuly_isJan(mes):
    if(int(mes) == 7 or int(mes) == 1):
        return 1
    else:
        return 0

In [91]:
param_grid = {
    'hidden_layer_sizes':[(5,),(10,),(20,)], # Feito # OK
    'activation':['logistic'], #Feito #OK
    'solver':['sgd'], # OK 
    'momentum':[0.5], 
    'learning_rate_init':[0.1,0.05],  # Feito # OK
    'max_iter': [1000,10000], #Feito #OK 
    'learning_rate':['constant'] #Valor default #Feito 
}

In [92]:
def diaDaSemana_isJanisJuly(dataFrame):
    dataFrame['seg'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 2 else 0 ,axis=1)
    dataFrame['ter'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 3 else 0 ,axis=1)
    dataFrame['quar'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 4 else 0 ,axis=1)
    dataFrame['qui'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 5 else 0 ,axis=1)
    dataFrame['sex'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 6 else 0 ,axis=1)
    dataFrame['sab'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 7 else 0 ,axis=1)
    dataFrame['dom'] = dataFrame.apply(lambda row: 1 if row['diaDaSemana'] == 1 else 0 ,axis=1)
    dataFrame['isJan_isJuly'] = dataFrame.apply(lambda row: isJuly_isJan(row['mes']),axis=1)
    return dataFrame

In [93]:
def onlyEquipament(equipamento,row):
    if(equipamento == row.equipamento):
        dataFrameTeste.append(row)

In [94]:
dfProcess = diaDaSemana_isJanisJuly(dfProcess)

In [20]:
for i in arrayEquipamento:
    print("Pegando valores do equipamento " + i )
    equipamento = dfProcess.loc[dfProcess['equipamento'] == i]
    equipamento = pd.DataFrame(equipamento)
    print("Min/Max Velo media e Hora")
    min_velocidadeMedia = equipamento['velocidadeMedia'].min()
    max_velocidadeMedia = equipamento['velocidadeMedia'].max()
    min_hora = equipamento['hora'].min()
    max_hora = equipamento['hora'].max()
    print("Normalizando dados, VeloMedia, Hora")
    print("Normalizando Velocidade Media")
    equipamento['velocidadeMedia'] = equipamento.apply(lambda row: valueToZeroToOne_velocidadeMedia(row['velocidadeMedia']),axis=1)
    equipamento['hora'] = equipamento.apply(lambda row: valueToZeroToOne_hora(row['hora']),axis=1)
    equipamento.velocidadeMedia = np.round(equipamento.velocidadeMedia,decimals=1)
    equipamento.hora = np.round(equipamento.hora,decimals=1)
    print(equipamento.hora)
    print(equipamento.velocidadeMedia)
    print("Parte do treinamento")
    df_x = np.asarray(equipamento[['seg','ter','quar','qui','sex','sab','dom','hora','isJan_isJuly']], dtype='float64')
    df_y = np.asarray(equipamento['velocidadeMedia'], dtype='float64')
    df_x = np.nan_to_num(df_x)
    df_y = np.nan_to_num(df_y)
    df_x = df_x.astype(np.float64)
    df_y = df_y.astype(np.float64)
    x_train, x_test, y_train, y_test = train_test_split(df_x,df_y, test_size=0.30)
    nn = MLPRegressor()
    gridSearch = GridSearchCV(nn,param_grid,cv=10,refit=False,scoring=['neg_mean_squared_error','neg_mean_squared_log_error','neg_median_absolute_error','neg_mean_absolute_error','neg_root_mean_squared_error'])
    print("Fazendo o FIT com o DataFrame")
    gridSearch = gridSearch.fit(x_train,y_train)
    print("Salvando dados no disco")
    mlpGridResult = pd.DataFrame(data=gridSearch.cv_results_)
    mlpGridResult.to_csv("result_" + i + ".csv",encoding='utf-8')

Pegando valores do equipamento FS002
Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
9343713    1.0
9343714    1.0
9343715    1.0
9343716    1.0
9343717    1.0
Name: hora, Length: 346968, dtype: float64
0          0.1
1          0.1
2          0.1
3          0.0
4          0.1
          ... 
9343713    0.1
9343714    0.1
9343715    0.1
9343716    0.1
9343717    0.1
Name: velocidadeMedia, Length: 346968, dtype: float64
Parte do treinamento
Fazendo o FIT com o DataFrame
Salvando dados no disco
Pegando valores do equipamento FS003
Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
7973       0.0
7974       0.0
7975       0.0
7976       0.0
7977       0.0
          ... 
9349445    1.0
9349446    1.0
9349447    1.0
9349448    1.0
9349449    1.0
Name: hora, Length: 239048, dtype: float64
7973       0.2
7974       0.2
79

Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
116171     0.0
116172     0.0
116173     0.0
116174     0.0
116175     0.0
          ... 
9454871    0.6
9454872    0.6
9454873    0.7
9454874    0.7
9454875    0.7
Name: hora, Length: 469385, dtype: float64
116171     0.4
116172     0.4
116173     0.4
116174     0.4
116175     0.4
          ... 
9454871    0.2
9454872    0.2
9454873    0.3
9454874    0.3
9454875    0.3
Name: velocidadeMedia, Length: 469385, dtype: float64
Parte do treinamento
Fazendo o FIT com o DataFrame
Salvando dados no disco
Pegando valores do equipamento FS016
Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
127665     0.0
127666     0.0
127667     0.0
127668     0.0
127669     0.0
          ... 
9462841    1.0
9462842    1.0
9462843    1.0
9462844    1.0
9462845    1.0
Name: hora, Length: 359007, dtype: float64
127665     0.6
127666     0.6
127667     0.6
127668     0.5
127669   

Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
4533421    0.7
4533422    0.7
4533423    0.7
4533424    0.7
4533425    0.7
          ... 
9535560    1.0
9535561    1.0
9535562    1.0
9535563    1.0
9535564    1.0
Name: hora, Length: 121804, dtype: float64
4533421    0.4
4533422    0.3
4533423    0.4
4533424    0.3
4533425    0.3
          ... 
9535560    0.2
9535561    0.3
9535562    0.3
9535563    0.2
9535564    0.2
Name: velocidadeMedia, Length: 121804, dtype: float64
Parte do treinamento
Fazendo o FIT com o DataFrame
Salvando dados no disco
Pegando valores do equipamento FS029
Min/Max Velo media e Hora
Normalizando dados, VeloMedia, Hora
Normalizando Velocidade Media
4534065    0.7
4534066    0.7
4534067    0.7
4534068    0.7
4534069    0.7
          ... 
9541497    1.0
9541498    1.0
9541499    1.0
9541500    1.0
9541501    1.0
Name: hora, Length: 122863, dtype: float64
4534065    0.5
4534066    0.5
4534067    0.4
4534068    0.4
4534069  